Learning LLM from scratch

In [45]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown,display
from openai import OpenAI



load .env file that we created and validate it has necessary keys


In [46]:
load_dotenv()
api_key=os.getenv("OPENAI_API_KEY")

# Check api key
if not api_key:
    print("No api key found , please check .env file")
elif api_key[:8]!="sk-proj-":
    
    print("An Api key found but it does not start with sk-proj-")
elif api_key.strip() != api_key:
    print("Api key exist but probably has extrachracters attached")
else:
    print("Api key found")

Api key found


In [47]:
openai = OpenAI()


Scrape test from a website


In [48]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    def __init__(self,url) -> None:
        self.url = url
        respose = requests.get(self.url,headers=headers)
        soup = BeautifulSoup(respose.content,'html.parser')
        self.title = soup.title.string if soup.title else "no title found"
        for irralavent in soup.body(["scripts","style","img","input"]):
            irralavent.decompose()
        self.text = soup.body.get_text(separator="\n",strip=True)
        

# ed = Website("https://edwarddonner.com")
# print(ed.title)
# print(ed.text)

Setting up System and user prompts

In [49]:
system_prompt ="you are an assistant that analyzes the contents of a website \
    and provides a short summary, ignoring text that might be navigation related. \
    Respond in markdown"

def user_prompt_for(website):
    user_prompt = f"you are looking at a website titled {website.title}"
    user_prompt += "\nthe content of the website  is as follow; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

The API from openAI expects to recieve message in a particular structure. Many of the other APIs share this structure:

```

```
[
    {"role":"system","content":"system message goes here"}
    {"role":"user","content":"user message goes here"}
]
```


In [50]:
def message_for(website):
    return [
         {"role":"system","content":system_prompt},
         {"role":"user","content":user_prompt_for(website)}]

In [51]:
message_for(ed)

NameError: name 'ed' is not defined

In [52]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages = message_for(website)
    )
    return response.choices[0].message.content

In [53]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [54]:
#display_summary("https://edwarddonner.com")
#display_summary("https://cnn.com")
display_summary("https://anthropic.com")

# Summary of Anthropic Website

Anthropic is focused on developing AI technologies that prioritize safety and human benefit. The website highlights their AI model, Claude, with the recent introduction of **Claude Opus 4** and **Claude Sonnet 4**, which are their most advanced models to date, improving capabilities in coding and AI-driven tasks.

## Key Features and Offerings
- **Claude AI Models**: Information about different models, including Opus 4 and Sonnet 4, showcasing advancements in AI technology.
- **API and Development Resources**: Tools and documentation for developers to build applications using Claude.
- **Collaborative Solutions**: Applications of Claude in various sectors such as customer support, education, and financial services.
- **Research Initiatives**: Commitment to AI safety through responsible scaling and transparency.

## News and Announcements
- **ISO 42001 Certification**: Announcement indicating the company’s compliance and commitment to high standards in AI practices.
- **Recent Articles**: Topics discussed include "Agentic Misalignment," "Tracing the Thoughts of a Large Language Model," and the **Anthropic Economic Index**, reflecting their ongoing research efforts and developments in the field of AI.

Overall, Anthropic strives to lead in responsible AI development while providing effective tools for various applications.